In [1]:
import os
import sys
import copy
import pickle
import numpy as np
import scipy as sci
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
from astropy import units 
import ChiantiPy.core as ch
from astropy.io import ascii
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.table import Column
import astropy.constants as const
from dynesty import utils as dyfunc
from dynesty.plotting import _quantile
from scipy.interpolate import interp1d
from dynesty import DynamicNestedSampler

from lmfit.models import Model
from specutils import Spectrum1D, SpectralRegion
from specutils.analysis import line_flux, equivalent_width
from astropy.nddata import StdDevUncertainty, NDUncertainty


%load_ext autoreload
%autoreload 2

sys.path.append('/Users/arcticfox/Documents/GitHub/cos_flares/src/')
from dem_modeling import ChiantiSetup, DEMModeling
from utils import *
from cos_flares import FlaresWithCOS

 ChiantiPy version 0.11.0 
 found PyQt5 widgets
 using PyQt5 widgets


/Users/arcticfox/opt/anaconda3/lib/python3.9/site-packages/ChiantiPy/core/IpyMspectrum.py:9: UserWarning: ipyparallel not found. You won't be able to use the ipymspectrum module
  warnings.warn("ipyparallel not found. You won't be able to use the ipymspectrum module")


In [2]:
def load_data(path):
    wavelength, flux, err = load_binned_data(os.path.join(path,'interp_between_1.npy'))

    times = np.load(os.path.join(path,'times.npy'))
    orbits = np.load(os.path.join(path,'orbits.npy'))

    wavelength = np.full(flux.shape, wavelength)
    return wavelength, flux, err, times, orbits

w20, f20, e20, t20, o20 = load_data(path='/Users/arcticfox/Documents/AUMic/30s/reduced')
w30, f30, e30, t30, o30 = load_data(path='/Users/arcticfox/Documents/AUMic/30s/reduced2')

_, flare_orbits, flare_mask = np.load('/Users/arcticfox/Documents/AUMic/flare_mask2.npy',
                                      allow_pickle=True)

In [3]:
fwc = FlaresWithCOS(wavelength=np.append(w20, w30, axis=0), 
                    flux=np.append(f20, f30, axis=0),
                    flux_err=np.append(e20, e30, axis=0), 
                    time=np.append(t20, t30)-t20[0], 
                    orbit=np.append(o20, o30+5))
fwc.load_line_table(path='/Users/arcticfox/Documents/GitHub/cos_flares/')

In [4]:
line_table = Table.read('/Users/arcticfox/Documents/AUMic/aumic_linelist.csv',
                        format='csv', comment='#')

In [5]:
mq = np.nansum(flare_mask,axis=0)==0
mean_quiescent = np.nanmean(fwc.flux[mq],axis=0)
mean_quiexcent_err = np.sqrt(np.nansum(fwc.flux_err[mq]**2, axis=0))/len(fwc.flux_err[mq])

## Creating the line list dictionary

Necessities: 
- 'centers' (center wavelength) --> done by setting vmin and vmax by-eye
- 'log10SFline' (surface flux in erg/cm$^2$/s)
- 'log10SFlineErr' (surface flux error in erg/cm$^2$/s)

In [28]:
aumic_linelist = {}
scaling = 1e-14

for i in range(len(line_table)):
    if line_table['quality'][i] == 0:
        main_key = line_table['Ion'][i].replace(' ', '')
        subdict = {}

        subdict['centers'] = np.array([line_table['wave_obs'][i]])

        q = ( (fwc.wavelength[0] >= line_table['wmin'][i]) &
              (fwc.wavelength[0] <= line_table['wmax'][i]) )

        subdict['lineX'] = fwc.wavelength[0][q]
        subdict['lineY'] = mean_quiescent[q]
        subdict['lineYerr'] = mean_quiexcent_err[q]
        
        ####################
        ##### line fit #####
        ####################
        gmodel = Model(gaussian, prefix='g_')
        pars = gmodel.make_params()
        pars['g_mu'].set(value=line_table['wave_obs'][i], 
                                           min=fwc.wavelength[0][q][5],
                                           max=fwc.wavelength[0][q][-5])
        pars['g_std'].set(value=0.1, min=0.01, max=20)
        pars['g_f'].set(value=0.5, min=0.01, max=40)
        init = gmodel.eval(pars, x=fwc.wavelength[0][q])
        out  = gmodel.fit(mean_quiescent[q]/scaling, 
                          pars,
                          x=fwc.wavelength[0][q])#.minimize(max_nfev=2000)
        mini = out.minimize(max_nfev=2000)
        
        upp = gaussian(fwc.wavelength[0][q], mini.params['g_mu'].value, mini.params['g_std'].value,
                       mini.params['g_f'].value+mini.params['g_f'].stderr)
        low = gaussian(fwc.wavelength[0][q], mini.params['g_mu'].value, mini.params['g_std'].value,
                       mini.params['g_f'].value-mini.params['g_f'].stderr)
        amp_std = np.nanmedian([np.nanmax(upp)-np.nanmax(out.best_fit),
                                np.nanmax(out.best_fit)-np.nanmax(low)])
        
        subdict['lineFit'] = out.best_fit*scaling
        subdict['amp'] = np.nanmax(out.best_fit)*scaling
        subdict['ampErr'] = amp_std*scaling
        subdict['fittedCenter'] = mini.params['g_mu'].value
        subdict['sig'] = mini.params['g_std'].value
        subdict['sigErr'] = mini.params['g_std'].stderr
        
        s1d = Spectrum1D(spectral_axis=subdict['lineX']*units.AA, 
                   flux=subdict['lineY']*units.erg/units.AA/units.s/units.cm**2, 
                   uncertainty=StdDevUncertainty(subdict['lineYerr']*units.erg/units.AA/units.s/units.cm**2))
        s2d = Spectrum1D(spectral_axis=subdict['lineX']*units.AA, 
                   flux=subdict['lineFit']*units.erg/units.AA/units.s/units.cm**2, 
                   uncertainty=StdDevUncertainty(subdict['lineYerr']*units.erg/units.AA/units.s/units.cm**2))
        
        subdict['Fline'] = line_flux(s1d)
        subdict['Ffit'] = line_flux(s2d)
        subdict['FlineErr'] = line_flux(s1d).uncertainty
        subdict['FfitErr'] = line_flux(s2d).uncertainty
        
        subdict['EW'] = equivalent_width(s1d)
        
        subdict['log10Fline'] = 0.0
        subdict['log10FlineErr'] = 0.0

        aumic_linelist[main_key] = subdict

In [29]:
aumic_linelist

{'NII': {'centers': array([1085.66418419]),
  'lineX': array([1085.31024401, 1085.31988689, 1085.32952977, 1085.33917265,
         1085.34881553, 1085.35845841, 1085.36810129, 1085.37774417,
         1085.38738705, 1085.39702994, 1085.40667282, 1085.4163157 ,
         1085.42595858, 1085.43560146, 1085.44524434, 1085.45488722,
         1085.4645301 , 1085.47417298, 1085.48381586, 1085.49345874,
         1085.50310162, 1085.5127445 , 1085.52238738, 1085.53203027,
         1085.54167315, 1085.55131603, 1085.56095891, 1085.57060179,
         1085.58024467, 1085.58988755, 1085.59953043, 1085.60917331,
         1085.61881619, 1085.62845907, 1085.63810195, 1085.64774483,
         1085.65738771, 1085.6670306 , 1085.67667348, 1085.68631636,
         1085.69595924, 1085.70560212, 1085.715245  , 1085.72488788,
         1085.73453076, 1085.74417364, 1085.75381652, 1085.7634594 ,
         1085.77310228, 1085.78274516, 1085.79238804, 1085.80203093,
         1085.81167381, 1085.82131669, 1085.830959

In [ ]:
lines = pd.read_pickle(r'/Users/arcticfox/Documents/AUMic/all_lines_transitions.pkl')
lines.keys()

In [ ]:
plt.plot(lines['SiIII']['LSF'][0])
plt.plot(lines['CIII']['LSF'][0])
plt.plot(lines['NV']['LSF'][0])

## Setting up files from Chianti

In [ ]:
lines = pd.read_pickle(r'/Users/arcticfox/Documents/AUMic/all_lines_transitions.pkl')
lines = lines.pop('Lya')

In [ ]:
cs = ChiantiSetup(linelist=lines,
                  logT_range=[4,8],
                  wave_range=[1,3200])

In [ ]:
cs.emissivity_functions()

In [ ]:
cs.get_all_ions()

In [ ]:
pickle.dump(cs.G_T, open('G_T.pkl','wb'))
pickle.dump(cs.linelist, open('linelist_updated.pkl', 'wb'))

## DEM Modeling Class

In [ ]:
%load_ext autoreload
%autoreload 2

sys.path.append('/Users/arcticfox/Documents/GitHub/cos_flares/src/')
from dem_modeling import ChiantiSetup, DEMModeling

In [ ]:
lines = pd.read_pickle(r'linelist_updated.pkl')
lines.pop('Lya');
G_T = pd.read_pickle(r'G_T.pkl')

In [ ]:
lines

In [ ]:
dem = DEMModeling(linelist=lines,
                  G_T=cs.G_T,
                  logT_range=cs.logT_range)

In [ ]:
dem.create_DEM(quick_plot=True, specified_lines=list(dem.linelist.keys())[:-2])

In [ ]:
dem.estimate_EUV_from_DEM()